In [ ]:
import pretty_midi
from corr_mat import *
from track import Track
import os
from tqdm import tqdm

In [ ]:
test_file = "./test_midi.mid"
pm = pretty_midi.PrettyMIDI(test_file)

In [ ]:
final_loops = []
for instrument in pm.instruments:
    track = Track(pm, instrument)
    note_list = track.notes
    lead_mat, lead_dur = calc_correlation(note_list)
    loops, loop_endpoints = get_valid_loops(track, lead_mat, lead_dur)
    print(f"found {len(loops)} loops in {instrument}")
    if len(loop_endpoints) > 0:
        print(loop_endpoints)
    for loop in loops:
        final_loops.append(loop)
print(f"{len(final_loops)} total loops in {len(pm.instruments)} tracks")

In [ ]:
directory = "/Users/sadkins/Downloads/Final_GigaMIDI_TISMIR/Test-10%/GigaMIDI-Test-Drum+Music-MD5"

def get_num_loops(pm):
    total_loops = 0
    for instrument in pm.instruments:
        track = Track(pm, instrument)
        note_list = track.notes
        lead_mat, lead_dur = calc_correlation(note_list)
        loops, loop_endpoints = get_valid_loops(track, lead_mat, lead_dur)
        total_loops += len(loop_endpoints)
    return total_loops, len(pm.instruments)

In [6]:
total_loops = 0
total_tracks = 0
num_files = 500
for file in tqdm(os.listdir(directory)[:num_files]):
    full_path = os.path.join(directory, file)
    pm = pretty_midi.PrettyMIDI(full_path)
    num_loops, num_tracks = get_num_loops(pm)
    total_loops += num_loops
    total_tracks += num_tracks
total_loops = 0
total_tracks = 0
num_files = 500
for file in tqdm(os.listdir(directory)[:num_files]):
    full_path = os.path.join(directory, file)
    pm = pretty_midi.PrettyMIDI(full_path)
    num_loops, num_tracks = get_num_loops(pm)
    total_loops += num_loops
    total_tracks += num_tracks
print(f"Found {total_loops} loops in {num_tracks} tracks across {num_files} files")